In [1]:
import plotly.express as px
import pandas as pd

In [2]:
df = pd.read_csv('conferencemap.csv')

print(f"Map created with {len(df)} academic events from {df['Year'].min()} to {df['Year'].max()}")
print(f"Countries represented: {', '.join(sorted(df['Country'].unique()))}")
df

Map created with 23 academic events from 2017 to 2026
Countries represented:  UK, France, Germany, Ireland, Lithuania, Spain, UK


,City,Country,Latitude,Longitude,Event,Year,Date,Full_Title,status
0,Banyuls-sur-Mer,France,42.4817,3.13170,École thématique d'été « Annotations »,2022,2022-06-01,École thématique d'été « Annotations »,accepted
1,Vilnius,Lithuania,54.6872,25.27970,ICAME 46,2025,2025-06-01,The 46th Annual Conference of the Internationa...,accepted
2,Mainz,Germany,49.9929,8.24730,Brexit means Brexit? Ein Symposium,2017,2017-12-01,Brexit means Brexit? Ein Symposium - Akademie ...,accepted
3,Lancaster,UK,54.0098,-2.78750,BAAL Corpus Linguistics SIG Workshop,2019,2019-11-01,BAAL Corpus Linguistics Special Interest Group...,accepted
4,Grenoble,France,45.1885,5.72450,Advanced Language Processing Winter School,2021,2021-01-01,Advanced Language Processing Winter School - ALPS,accepted
5,Lancaster,UK,54.0098,-2.78750,Machine Learning for Humanists,2021,2021-06-01,Machine Learning for Humanists - N8 CIR,accepted
6,Limerick,Ireland,52.6638,-8.62670,CL 2021,2021,2021-07-01,The International Corpus Linguistics Conferenc...,accepted
7,Cambridge,UK,52.2055,0.12100,ICAME 43,2022,2022-07-01,The 43rd Annual Conference of the Internationa...,accepted
8,London,UK,51.5074,-0.12780,CHIMED-3,2023,2023-05-01,"CHIMED-3, 3rd International Conference on Hist...",accepted
9,Lancaster,UK,54.0098,-2.78750,CL2023,2023,2023-07-01,"CL2023, 12th International Corpus Linguistics ...",accepted


In [3]:
# Create a new column for location
df['Location'] = df['City'] + ', ' + df['Country']
# Create a new column for event count
df['Event Count'] = df.groupby('Location')['Event'].transform('count')

# Group events by location and concatenate event titles
df_grouped = df.groupby(['Location', 'Latitude', 'Longitude'])['Event'].apply(lambda x: '<br> '.join(x)).reset_index()
df_years = df.groupby(['Location', 'Latitude', 'Longitude'])['Year'].apply(lambda x: ', '.join(map(str, x))).reset_index()

# Merge the grouped data with the original data
df = df.merge(df_grouped, on=['Location', 'Latitude', 'Longitude'], suffixes=('', '_grouped'))
df = df.merge(df_years, on=['Location', 'Latitude', 'Longitude'], suffixes=('', '_years'))
df['Is_Past'] = df['Date'].apply(lambda x: pd.to_datetime(x) < pd.Timestamp('today'))
df['Year_years'] = df['Year_years'].apply(lambda x: ','.join(sorted(set(x.split(',')), key=int)))

In [4]:


# Create the map
fig = px.scatter_map(
    df,
    lat='Latitude',
    lon='Longitude',
    hover_name='Location',
    hover_data={
        'Location': True,
        'Event_grouped': True,
        'Year_years': False
    },
    size='Event Count',
    width=800,
    height=600,
    zoom=3.2,
    map_style= 'carto-positron'
    )

# Update marker styling
fig.update_traces(
    marker=dict(
        opacity=0.55,
        color=[ "#655EC5" if x else "#35eb44" if df.loc[i, "status"] != "accepted" else "#cbd338" for i, x in df["Is_Past"].items()]
    ),
    hovertemplate='<b>%{customdata[0]}</b>'+
                  '<br>%{customdata[1]}' +
                  '<br>%{customdata[2]}'
)

# Update layout for better aesthetics
fig.update_layout(
    font=dict(size=12),
    margin=dict(l=0, r=0, t=0, b=0),
    showlegend=False  # Hide the legend
)

# Show the map
fig.show()
fig.write_html("conferencemap.html")